In [1]:
import sys

In [2]:
import os
import stellargraph as sg
import tensorflow as tf
import pandas as pd
import os
import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GCN

from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline
from spektral.layers import GCNConv, GlobalSumPool

2023-03-14 09:54:40.818993: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 09:54:42.146197: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46723 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:31:00.0, compute capability: 8.6
2023-03-14 09:54:42.147430: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 567 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]='1'

In [4]:
dataset = sg.datasets.Cora()

In [5]:
display(HTML(dataset.description))
G, node_subjects = dataset.load()

In [6]:
train_subjects, test_subjects = model_selection.train_test_split(
    node_subjects, train_size=140, test_size=None, stratify=node_subjects
)
val_subjects, test_subjects = model_selection.train_test_split(
    test_subjects, train_size=500, test_size=None, stratify=test_subjects
)

In [7]:
train_subjects.value_counts().to_frame()

,subject
Neural_Networks,42
Genetic_Algorithms,22
Probabilistic_Methods,22
Theory,18
Case_Based,16
Reinforcement_Learning,11
Rule_Learning,9


In [8]:
target_encoding = preprocessing.LabelBinarizer()

train_targets = target_encoding.fit_transform(train_subjects)
val_targets = target_encoding.transform(val_subjects)
test_targets = target_encoding.transform(test_subjects)

In [9]:
generator = FullBatchNodeGenerator(G, method="gcn")

Using GCN (local pooling) filters...


In [10]:
train_gen = generator.flow(train_subjects.index, train_targets)

In [11]:
class ProteinsGNN(Model):
 
  def __init__(self, n_hidden, n_labels):
    super().__init__()
    # Define our GCN layer with our n_hidden layers
    self.graph_conv = GCNConv(n_hidden)
    # Define our global pooling layer
    self.pool = GlobalSumPool()
    # Define our dropout layer, initialize dropout freq. to .5 (50%)
    #self.dropout = Dropout(0.5)
    # Define our Dense layer, with softmax activation function
    self.dense = tf.keras.layers.Dense(n_labels, 'softmax')
 
# Define class method to call model on input
  def call(self, inputs):
    out = self.graph_conv(inputs)
    out = self.dropout(out)
    out = self.pool(out)
    out = self.dense(out)
 
    return out

In [12]:
model = ProteinsGNN(32, 7)

In [13]:
gcn = GCN(
    layer_sizes=[16, 16], activations=["relu", "relu"], generator=generator, dropout=0.5
)
x_inp, x_out = gcn.in_out_tensors()

x_out
predictions = layers.Dense(units=train_targets.shape[1], activation="softmax")(x_out)

In [14]:
model = Model(inputs=x_inp, outputs=predictions)
model.compile(
    optimizer=optimizers.Adam(lr=0.01),
    loss=losses.categorical_crossentropy,
    metrics=["acc"],
)

/home/jhjoo/anaconda3/envs/ecg/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [15]:
val_gen = generator.flow(val_subjects.index, val_targets)

In [16]:
from tensorflow.keras.callbacks import EarlyStopping

es_callback = EarlyStopping(monitor="val_acc", patience=50, restore_best_weights=True)

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(1, 2708, 1433)]    0           []                               
                                                                                                  
 input_3 (InputLayer)           [(1, None, 2)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(1, None)]          0           []                               
                                                                                                  
 dropout (Dropout)              (1, 2708, 1433)      0           ['input_1[0][0]']                
                                                                                              

In [18]:
history = model.fit(
    train_gen,
    epochs=200,
    validation_data=val_gen,
    verbose=2,
    shuffle=False,  # this should be False, since shuffling data means shuffling the whole graph
    callbacks=[es_callback],
)

Epoch 1/200


2023-03-14 09:54:46.418144: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 - 3s - loss: 1.9535 - acc: 0.1429 - val_loss: 1.9151 - val_acc: 0.1740 - 3s/epoch - 3s/step
Epoch 2/200
1/1 - 0s - loss: 1.9007 - acc: 0.1857 - val_loss: 1.8764 - val_acc: 0.1800 - 164ms/epoch - 164ms/step
Epoch 3/200
1/1 - 0s - loss: 1.8448 - acc: 0.2143 - val_loss: 1.8330 - val_acc: 0.1740 - 143ms/epoch - 143ms/step
Epoch 4/200
1/1 - 0s - loss: 1.7902 - acc: 0.1929 - val_loss: 1.7883 - val_acc: 0.1700 - 151ms/epoch - 151ms/step
Epoch 5/200
1/1 - 0s - loss: 1.7289 - acc: 0.2286 - val_loss: 1.7419 - val_acc: 0.1860 - 167ms/epoch - 167ms/step
Epoch 6/200
1/1 - 0s - loss: 1.6544 - acc: 0.2357 - val_loss: 1.6938 - val_acc: 0.2260 - 158ms/epoch - 158ms/step
Epoch 7/200
1/1 - 0s - loss: 1.6019 - acc: 0.3071 - val_loss: 1.6435 - val_acc: 0.2900 - 177ms/epoch - 177ms/step
Epoch 8/200
1/1 - 0s - loss: 1.5281 - acc: 0.3286 - val_loss: 1.5924 - val_acc: 0.3800 - 163ms/epoch - 163ms/step
Epoch 9/200
1/1 - 0s - loss: 1.4793 - acc: 0.4000 - val_loss: 1.5401 - val_acc: 0.4720 - 156ms/epoch - 156

Epoch 73/200
1/1 - 0s - loss: 0.1149 - acc: 0.9643 - val_loss: 0.9964 - val_acc: 0.7920 - 144ms/epoch - 144ms/step
Epoch 74/200
1/1 - 0s - loss: 0.0554 - acc: 0.9857 - val_loss: 0.9903 - val_acc: 0.7920 - 144ms/epoch - 144ms/step
Epoch 75/200
1/1 - 0s - loss: 0.0451 - acc: 0.9929 - val_loss: 0.9787 - val_acc: 0.7900 - 157ms/epoch - 157ms/step
Epoch 76/200
1/1 - 0s - loss: 0.0313 - acc: 0.9929 - val_loss: 0.9691 - val_acc: 0.7920 - 162ms/epoch - 162ms/step
Epoch 77/200
1/1 - 0s - loss: 0.0509 - acc: 0.9857 - val_loss: 0.9643 - val_acc: 0.7860 - 165ms/epoch - 165ms/step
Epoch 78/200
1/1 - 0s - loss: 0.0322 - acc: 0.9929 - val_loss: 0.9615 - val_acc: 0.7860 - 150ms/epoch - 150ms/step
Epoch 79/200
1/1 - 0s - loss: 0.0506 - acc: 0.9786 - val_loss: 0.9622 - val_acc: 0.7880 - 156ms/epoch - 156ms/step
Epoch 80/200
1/1 - 0s - loss: 0.0521 - acc: 0.9857 - val_loss: 0.9669 - val_acc: 0.7900 - 171ms/epoch - 171ms/step
Epoch 81/200
1/1 - 0s - loss: 0.0513 - acc: 0.9857 - val_loss: 0.9753 - val_acc:

In [19]:
test_gen = generator.flow(test_subjects.index, test_targets)

In [20]:
test_metrics = model.evaluate(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

1/1 [==============================] - 0s 325ms/step - loss: 0.9974 - acc: 0.7843

Test Set Metrics:
	loss: 0.9974
	acc: 0.7843
